In [1]:
import itertools

import numpy as np
import pandas as pd

from PIL import Image

In [2]:
# Transforms an RGB matrix of pixels to a Grayscale matrix of pixels
#[CITE] https://stackoverflow.com/q/12201577
def rgb2gray(rgb):
    r, g, b = rgb[:,:,0], rgb[:,:,1], rgb[:,:,2]
    gray = 0.2989 * r + 0.5870 * g + 0.1140 * b
    return gray

# Convert a array of lists to a singular list
#[CITE] https://stackoverflow.com/a/716482
def listsToList(listOfLists):
    return list(itertools.chain.from_iterable(listOfLists))

# Converts a image to a grayscale singular array of pixels
def imageToArray(imgPath):
    return listsToList(rgb2gray(np.asarray(Image.open(imgPath))))


In [3]:
# TODO: Greatly imporve how the images are retrieved
cats = []
dogs = []

for i in range(0,1250):
    cats.append(imageToArray(f'./train/cat.{i}.jpg'))
    dogs.append(imageToArray(f'./train/dog.{i}.jpg'))
    
animalClass = np.zeros(len(cats)+len(dogs))
animalClass[1250:] = 1
    
animals = {'ImageData': cats+dogs, 'ImageClass':animalClass}
# Get some memory back
cats =[]
dogs =[]

ds = pd.DataFrame(data=animals)
# Get some memory back
animals = []

In [5]:
ds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2500 entries, 0 to 2499
Data columns (total 2 columns):
ImageClass    2500 non-null float64
ImageData     2500 non-null object
dtypes: float64(1), object(1)
memory usage: 39.1+ KB
